In [21]:
import time
import sqlite3
import requests
import datetime

import pandas as pd
import shioaji as sj
import re
from shioaji import TickFOPv1, Exchange
from exchange_calendars import get_calendar

import mplfinance as mpf


In [22]:
import os 
import shioaji as sj
from dotenv import load_dotenv

#永豐金API登入
load_dotenv()
api = sj.Shioaji(simulation=True)
api.login(os.getenv('YOUR_PERSON_ID'),os.getenv('YOUR_PASSWORD'),
    contracts_cb= lambda security_type: print(f"{repr(security_type)} fetch done"))

#連結DB Browser
connection = sqlite3.connect('work_data.db')

Response Code: 0 | Event Code: 0 | Info: host '203.66.91.161:80', hostname '203.66.91.161:80' IP 203.66.91.161:80 (host 1 of 1) (host connection attempt 1 of 1) (total connection attempt 1 of 1) | Event: Session up
<SecurityType.Index: 'IND'> fetch done
<SecurityType.Stock: 'STK'> fetch done
<SecurityType.Future: 'FUT'> fetch done
<SecurityType.Option: 'OPT'> fetch done


get_ticks() <br>
抓取指定日期及前三日中的Ticks並更新至資料庫 <br>

In [31]:
def get_ticks(connection, api, date, codes = str, is_Futures = False):
    print(date)

    dates = pd.to_datetime(date)
    tw_calendar = get_calendar('XTAI')
    if date not in tw_calendar.opens:
        print('{} is not trading date'.format(date))

    if is_Futures:
        
        #若為期貨只取英文代號作為code name
        code = ''.join(char for char in codes if char.isalpha())

        if dates.hour > 15:
            date = dates.date()+datetime.timedelta(days=1)
        
        else:
            date = dates.date()

        prev_trading_date = tw_calendar.sessions_window(date,-3)
        for trading_date in prev_trading_date:
            if trading_date.weekday() == 5:
                trading_dates = trading_date+datetime.timedelta(days=1)
                prev_trading_date = prev_trading_date.append(trading_dates)

    else:
        code = codes
        date = dates.date()
        
        prev_trading_date = tw_calendar.sessions_window(date,-3)


    #儲存 Total Data
    main_df = pd.DataFrame()

    for trading_date in prev_trading_date:
    #檢查是否為固有資料        
        try:        
            sql = "SELECT * FROM ticks WHERE code = '{}' and ts BETWEEN '{}' AND '{}' ".format(code,
                                                                                        trading_date,
                                                                                        trading_date+datetime.timedelta(days=1))
            df = pd.read_sql(sql, connection, parse_dates = ['ts'], index_col=['ts'])
        except:
            df = pd.DataFrame()

        #若已有資料，直接回傳dataframe    
        if (not df.empty):
            main_df = pd.concat([main_df, df], sort=False)
        
        else:
            print(f'Fetching missing data for {trading_date.strftime("%Y-%m-%d")}')

            #若資料不存在，利用永豐API獲取 
            if is_Futures: #若為期貨
                ticks = api.ticks(
                    contract = api.Contracts.Futures.get(code)[codes],  # For futures, use the Futures contract
                    date=date.strftime('%Y-%m-%d') 
                    )

            else: #若為證券
                ticks = api.ticks(
                    contract=api.Contracts.Stocks[codes],  # For stocks, use the Stocks contract
                    date=date.strftime('%Y-%m-%d')
                )

            temp_df = pd.DataFrame({**ticks})
            temp_df.ts = pd.to_datetime(temp_df.ts)
            temp_df['code'] = code
            temp_df = temp_df.set_index('ts')
        
            main_df = pd.concat([main_df, temp_df], sort= False)

    if not main_df.empty:
        main_df = main_df.reset_index().drop_duplicates(subset='ts').set_index('ts').sort_index()
        main_df = main_df.sort_index()
        main_df.to_sql('ticks', connection, if_exists='append')
        
    return main_df, False

In [32]:
codes = 'TMF202410'
date = pd.to_datetime('2024-10-2')

df = get_ticks(connection, api, date, codes, True)

2024-10-02 00:00:00


禮拜五下午的交易資料怎麼取得

In [12]:
#獲取ticks資料並更新資料庫(DB)
"""
def update_ticks(connection, api, date, codes = str, is_Futures = False):

    main_df = pd.DataFrame()

    dates = pd.to_datetime(date)
    tw_calendar = get_calendar('XTAI')

    if dates.hour > 15:
        date = dates.date()+datetime.timedelta(days=1)
        print(date)

    else:
        date = dates.date()


    df, in_db = get_ticks(connection, api, date, codes , is_Futures)
  
    if df is not None and not in_db:
        main_df = pd.concat([main_df, df], sort = False, axis = 0)
        time.sleep(1)
    
'''
    #好像跟get_ticks的功能重複
    prev_trading_date = tw_calendar.sessions_window(date,-3)
    for prev_date in prev_trading_date:
        print('正在更新{}:{}的逐筆成交資料'.format(prev_date.strftime('%Y-%m-%d'), codes))
        prev_df, prev_in_db = get_ticks(connection, api, prev_date, codes, is_Futures)

        if prev_df is not None and not prev_in_db:
            main_df = pd.concat([main_df, prev_df], sort = False)
            time.sleep(1)
'''
        
    if not main_df.empty:
        main_df = main_df.reset_index().drop_duplicates().set_index('ts')
        main_df = main_df.sort_index()
        main_df.to_sql('ticks', connection, if_exists='append')
        
    return main_df if not main_df.empty else None

    """

In [ ]:
codes = 'TMF202410'
date = '2024-10-01'

update_ticks(connection, api, date, codes, True)

In [ ]:
ticks

Kbars

In [7]:
#將ticks轉換成kbar
def ticks_to_kbars(ticks, interval = '30Min'):

    kbars = pd.DataFrame()

    kbars['open'] = ticks['close'].resample(interval, closed='right', label = 'left').first()
    kbars['close'] = ticks['close'].resample(interval, closed='right', label = 'left').last()
    kbars['high'] = ticks['close'].resample(interval, closed='right', label = 'left').max()
    kbars['low'] = ticks['close'].resample(interval, closed='right', label = 'left').min()
    kbars['volume'] = ticks['volume'].resample(interval, closed='right', label = 'left').sum()

    kbars.dropna(inplace=True)

    return kbars

In [15]:
code = 'TMF202410'
date = pd.to_datetime('2024/9/19')
ticks = get_ticks(connection, api, date, code , True)[0]
ticks
kbars=ticks_to_kbars(ticks, '30MIN')

In [ ]:
marketcolors = mpf.make_marketcolors(up = '#E9544E',down='#56B475', inherit = True)
style = mpf.make_mpf_style(base_mpf_style = 'yahoo', marketcolors = marketcolors)


mpf.plot(kbars, 
         title = '{}, {}'.format(code, date.date()), 
         type = 'candle',
         style = style,
         volume = True,
         figsize = (14,8)
)

即時報價

In [7]:
ticks = {}
kbars = {}
volume_today = {}

In [ ]:
ticks

In [9]:
import time

#tbot0: 更新當日及前三日ticks資料

def tbot0(codes, is_Futures = False):

    print('--- 啟動 Tbot for {} ---'.format(codes))


    tw_calendar = get_calendar('XTAI')

    date = pd.to_datetime(datetime.datetime.now().date())

    #若非交易日，直接回傳
    if date not in tw_calendar.opens:
        print('今日非交易日')
        return

    prev_trading_dates = tw_calendar.sessions_window(date, -1)[0]

    if is_Futures:
        code = ''.join(char for char in codes if char.isalpha())
    else:
        code = codes
    ticks[code] = get_ticks(connection, api, date, codes , is_Futures )[0]
    ticks[code] = ticks[code][['close','volume']]
    kbars[code] = ticks_to_kbars(ticks[code], interval='1Min')
    kbars[code] = kbars[code][prev_trading_dates:]
    volume_today[code] = kbars[code]['volume'].sum()

    api.quote.subscribe(api.Contracts.Futures.TMF['TMF202410'], quote_type='tick',
                    version = sj.constant.QuoteVersion.v1
    )

    return ticks


In [ ]:
code = 'TMF202410'
tbot0(code, True)

In [ ]:
ticks
kbars
#volume_today

In [ ]:
api.quote.subscribe(api.Contracts.Futures.TMF['TMF202410'], quote_type='tick',
                    version = sj.constant.QuoteVersion.v1
)

In [ ]:
ticks
#ticks[code]
kbars
#kbars[code]
#volume_today

In [ ]:
api.quote.unsubscribe(
    api.Contracts.Futures.TMF['TMF202410'],
    quote_type = sj.constant.QuoteType.Tick,
    version = sj.constant.QuoteVersion.v1
)

In [12]:
#設定callback 回傳資料型態
@api.on_tick_fop_v1()

def quote_callback(exchange: Exchange, tick: TickFOPv1):

    ts = pd.to_datetime(tick.datetime)

    if tick.code[0].isalpha():
        tick.datetime
        code = tick.code[0:3]
    else:
        code = tick.code

    ts = pd.to_datetime(tick.datetime)

    if 5 < ts.hour < 9:
        return
    
    close = float(tick.close)
    volume = tick.volume
    vol_sum = tick.total_volume

    
    ticks[code].loc[ts] = [close, volume]
    volume_today[code] = vol_sum

In [8]:
import time

#tbot: 加入即時訂閱，即時更新tick資料
def tbot(codes, is_Futures = False):

    print('--- 啟動 Tbot for {} ---'.format(codes))

    ticks = {}
    kbars = {}
    volume_today = {}

    tw_calendar = get_calendar('XTAI')

    date = pd.to_datetime(datetime.datetime.now().date())

    if date not in tw_calendar.opens:
        print('今日非交易日')
        return

    prev_trading_dates = tw_calendar.sessions_window(date, -4)[0]

    if is_Futures:
        code = ''.join(char for char in codes if char.isalpha())
    else:
        code = codes
    
    ticks[code] = get_ticks(connection, api, date, codes , is_Futures )[0]
    ticks[code] = ticks[code][['close','volume']]
    kbars[code] = ticks_to_kbars(ticks[code], interval='1Min')
    kbars[code] = kbars[code][prev_trading_dates:]
    volume_today[code] = kbars[code]['volume'].sum()
    print(kbars)


    if is_Futures:
        api.quote.subscribe(api.Contracts.Futures.get(codes[0:3])[codes], 
                            quote_type= 'tick',
                            version = sj.constant.QuoteVersion.v1 
                            )
        while True:

            time.sleep(1)

            current_time = datetime.datetime.now()

            if current_time.second == 0 :
                print(ticks)

                kbars[code] = ticks_to_kbars(ticks[code], interval='1Min')
                kbars[code] = kbars[code][prev_trading_dates:]

                str_current_time = (current_time - datetime.timedelta(minutes=1)).strftime('%Y-%m-%d %H:%M:%S')

                try:
                    current_kbars = kbars[code].loc[str_current_time]
                except:
                    continue

                print(current_kbars)

        

    else:
        api.quote.subscribe(api.Contracts.Stocks[codes],
                            quote_type= 'tick',
                            version = sj.constant.QuoteVersion.v1
                            )
    ##subscribed ticks can't concat into Original ticks !!!
        while True:

            time.sleep(1)

            current_time = datetime.datetime.now()

            if current_time.second == 0 :
                print(ticks)

                kbars[code] = ticks_to_kbars(ticks[code], interval='1Min')
                kbars[code] = kbars[code][prev_trading_dates:]

                str_current_time = (current_time - datetime.timedelta(minutes=1)).strftime('%Y-%m-%d %H:%M:%S')

                try:
                    current_kbars = kbars[code].loc[str_current_time]
                except:
                    continue

                print(current_kbars)


In [ ]:
tbot('TMF202410', True)

In [ ]:
api.quote.unsubscribe(
    api.Contracts.Futures.TMF['TMF202410'],
    quote_type = sj.constant.QuoteType.Tick,
    version = sj.constant.QuoteVersion.v1
)